In [68]:
from selenium import webdriver
import time
import pandas
import csv

In [69]:
details_mapping = {
    "COLOUR": "EXTERIOR COLOUR",
    "VARIANT": "BADGE",
    "KMS SHOWING": "ODOMETER",
    "MOTOR TYPE": "ENGINE",
    "GEARBOX MAKE/MODEL": "TRANSMISSION",
    "STEERING TYPE": "STEERING",
    "ENIGNE": "ENGINE NUMBER",}

In [70]:
RESEARCH_BASE_URL = "https://www.slatteryauctions.com.au/search/cars-commercial-vehicles-motorcycles?gclid=Cj0KCQjwl8XtBRDAARIsAKfwtxBX33_4QMER8CNoUeURBhw1Q4IStUCf7ORD8CnKeSoWvAW2wa5U3ToaApoqEALw_wcB"
EXECUTABLE_PATH = "/home/saronida/lib/geckodriver"

In [71]:
driver = webdriver.Firefox(executable_path=EXECUTABLE_PATH)

In [72]:
driver.get(RESEARCH_BASE_URL)

In [73]:
time.sleep(10)
item_cards = driver.find_elements_by_class_name("item-card__inner")

In [74]:
len(item_cards)

127

In [75]:
cars_urls = [item_card.find_element_by_tag_name("a").get_attribute("href") for item_card in item_cards]

In [76]:
first_example_url = "https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador"

In [83]:
first_example_url

'https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador'

In [84]:
driver.get(first_example_url)

In [85]:
link = first_example_url

In [86]:
title = driver.find_elements_by_class_name("product-details__title")[1].text

In [87]:
title

'2017 LAMBORGHINI AVENTADOR'

In [88]:
price = driver.find_element_by_class_name("starting-bid").find_element_by_class_name("ng-binding").text

In [89]:
price

'$300,000.00'

In [90]:
initial_details = {
    "TITLE": title, "LINK": link, "PRICE": price}

In [91]:
initial_details["DEALER NAME"] = "Slattery Auctions"

In [92]:
location = driver.find_element_by_class_name("snapshot__details").find_elements_by_tag_name("li")[2].text

In [93]:
location

'6-8 Waterview Close, Dandenong South VIC 3175'

In [94]:
initial_details["LOCATION"] = location

In [95]:
initial_details

{'TITLE': '2017 LAMBORGHINI AVENTADOR',
 'LINK': 'https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador',
 'PRICE': '$300,000.00',
 'DEALER NAME': 'Slattery Auctions',
 'LOCATION': '6-8 Waterview Close, Dandenong South VIC 3175'}

In [96]:
##############

In [97]:
product_details_comments_features = driver.find_element_by_id("product-details")

In [98]:
description = product_details_comments_features.find_elements_by_class_name("table-product-details")[0]

In [99]:
description_items = description.text.split("\n")

In [100]:
description_details = [item for item in description_items if ":" in item and len(item.split(":"))==2]

In [101]:
details_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[1]

In [102]:
details = details_block.find_elements_by_tag_name("tr")

In [103]:
len(details)

12

In [104]:
asset_conditions_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[2]

In [105]:
asset_conditions = asset_conditions_block.find_elements_by_tag_name("tr")

In [106]:
len(asset_conditions)

10

In [107]:
extra_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[3]

In [108]:
extra = extra_block.find_elements_by_tag_name("tr")

In [109]:
len(extra)

14

In [110]:
#main_pane = driver.find_element_by_id("product-details-main-pane")

In [111]:
#main_pane_details_block = main_pane.find_element_by_class_name("table-product-details")

In [112]:
#main_pane_details = main_pane_details_block.find_elements_by_tag_name("tr")

In [113]:
#extra[0].find_element_by_tag_name("td").find_element_by_tag_name("i").get_attribute("class")

In [114]:
def parse_details(description_details, details, asset_conditions, extra, initial_details):
    """ Parses vehicle's details box. """

    parsed_details = initial_details
    
    for detail in description_details:
        key = detail.split(":")[0]
        value = detail.split(":")[1]
        if value != "":
            parsed_details[key] = value
            
    for detail in details:
        parsed_details[detail.find_element_by_tag_name("th").text] = detail.find_element_by_tag_name("td").text
    
    for detail in asset_conditions:
        parsed_details[detail.find_element_by_tag_name("th").text] = detail.find_element_by_tag_name("td").text
    
    features_list = []
    for feature in extra:
        if feature.find_element_by_tag_name("td").find_element_by_tag_name("i").get_attribute("class"):
            features_list.append(feature.find_element_by_tag_name("th").text)
    parsed_details["VEHICLE FEATURES"] = ",".join(features_list)
    
    parsed_details_df = pandas.DataFrame.from_dict(parsed_details, orient="index", columns=["value"])
    parsed_details_df["key"] = parsed_details_df.index
    parsed_details_df.reset_index(inplace=True, drop=True)
    return parsed_details_df

In [115]:
parsed_details_df = parse_details(description_details, details, asset_conditions, extra, initial_details)

In [116]:
parsed_details_df

,value,key
0,2017 LAMBORGHINI AVENTADOR,TITLE
1,https://www.slatteryauctions.com.au/product/44...,LINK
2,"$300,000.00",PRICE
3,Slattery Auctions,DEALER NAME
4,"6-8 Waterview Close, Dandenong South VIC 3175",LOCATION
5,ZHWET3ZD9HLA06405,VIN
6,L53906796,ENGINE
7,GREY,COLOUR
8,7021,ODOMETER
9,01/06/2017,COMPLIANCE DATE


In [117]:
def alter_details(parsed_details_df):
    """ Alters details to match mapping format. """

    parsed_details_df = parsed_details_df[~pandas.isnull(parsed_details_df.key)]
    parsed_details_df["key"] = parsed_details_df["key"].apply(lambda key: key.replace(":", "").strip().upper())
    parsed_details_df["key"] = parsed_details_df["key"].apply(
        lambda key: details_mapping[key] if key in details_mapping.keys() else key)
    parsed_details_df.drop_duplicates(subset ="key", inplace = True)
    return parsed_details_df

In [118]:
parsed_details_df = alter_details(parsed_details_df)

In [119]:
tmp_dict = parsed_details_df.to_dict(orient="list")
parsed_details = dict(zip(tmp_dict["key"], tmp_dict["value"]))

In [120]:
parsed_details["VEHICLE FEATURES"]

'Key,A/C,ABS,Leather,Power Windows,Drivable,Central Locking,Cruise Control,Sun Roof,Power Mirrors,Logbook,Airbag(s),Alloys,CD Player'

In [121]:
#to_save_df

In [122]:
with open('./results/slattery.csv', 'w') as f:
    w = csv.DictWriter(f, parsed_details.keys())
    w.writeheader()
    w.writerow(parsed_details)